In [ ]:
!python -m pip install --upgrade setuptools
!pip install git+https://github.com/tky823/ssspy.git

In [ ]:
import numpy as np
import scipy.signal as ss
import matplotlib.pyplot as plt
import IPython.display as ipd

In [ ]:
from ssspy.utils.dataset import download_sample_speech_data

In [ ]:
n_sources = 4
max_duration = 10
sisec2010_tag = "dev1_female4"
n_fft, hop_length = 4096, 2048

In [ ]:
waveform_src_img, sample_rate = download_sample_speech_data(
    n_sources=n_sources,
    sisec2010_tag=sisec2010_tag,
    max_duration=max_duration,
    conv=True,
) # (n_channels, n_sources, n_samples)
waveform_mix = np.sum(waveform_src_img, axis=1)  # (n_channels, n_samples)

In [ ]:
for idx, waveform in enumerate(waveform_mix):
    print("Mixture: {}".format(idx + 1))
    display(ipd.Audio(waveform, rate=sample_rate))
    print()

In [ ]:
from ssspy.bss.ilrma import TILRMA

## w/ partitioning function

In [ ]:
ilrma = TILRMA(
    n_basis=8,
    dof=1000,
    spatial_algorithm="IP2",
    domain=2,
    partitioning=True, # w/ partitioning function
    rng=np.random.default_rng(42),
)
print(ilrma)

In [ ]:
_, _, spectrogram_mix = ss.stft(waveform_mix, window="hann", nperseg=n_fft, noverlap=n_fft-hop_length)

In [ ]:
spectrogram_est = ilrma(spectrogram_mix, n_iter=200)

In [ ]:
_, waveform_est = ss.istft(spectrogram_est, window="hann", nperseg=n_fft, noverlap=n_fft-hop_length)

In [ ]:
for idx, waveform in enumerate(waveform_est):
    print("Estimated source: {}".format(idx + 1))
    display(ipd.Audio(waveform, rate=sample_rate))
    print()

In [ ]:
plt.figure()
plt.plot(ilrma.loss[1:])
plt.show()
plt.close()

## w/o partitioning function

In [ ]:
ilrma = TILRMA(
    n_basis=2,
    dof=1000,
    spatial_algorithm="IP2",
    domain=2,
    partitioning=False, # w/o partitioning function
    rng=np.random.default_rng(42),
)
print(ilrma)

In [ ]:
_, _, spectrogram_mix = ss.stft(waveform_mix, window="hann", nperseg=n_fft, noverlap=n_fft-hop_length)

In [ ]:
spectrogram_est = ilrma(spectrogram_mix, n_iter=200)

In [ ]:
_, waveform_est = ss.istft(spectrogram_est, window="hann", nperseg=n_fft, noverlap=n_fft-hop_length)

In [ ]:
for idx, waveform in enumerate(waveform_est):
    print("Estimated source: {}".format(idx + 1))
    display(ipd.Audio(waveform, rate=sample_rate))
    print()

In [ ]:
plt.figure()
plt.plot(ilrma.loss[1:])
plt.show()
plt.close()